# First Test of The Model

Testing the model after training it on one of the forty files in the dataset. Unfortunately the trained model is takes about 1GB so  cannot upload it to github. 

In [37]:
import gensim
import pandas as pd

Loading the filtered dataset

In [38]:
data_dir = '../data/csvTrain/'
filename = 's2-corpus-002_lang.csv'

In [39]:
df = pd.read_csv(data_dir+filename)

Finding a paper from PRL (I have some domain knowledge in the field):

In [85]:
df[df.journalName.str.contains('review letters')].head()

,Unnamed: 0,id,paperAbstract,title,year,journalName,lang
1917,1917,c367c5744775fa3d79404e1a5ad8cb0f9107655f,The branching fraction ratio R(D^{*})≡B(B[over...,Measurement of the ratio of branching fraction...,2015.0,Physical review letters,English
2952,2952,3758ce746008b7d6ccc06d49e57a235a4c21c6f0,We grew tetragonally distorted FexCo1-x alloy ...,Perpendicular magnetic anisotropy induced by t...,2006.0,Physical review letters,English
6422,6422,202dd914cbaadbd5ee76c1ddc919750f949c669b,"We introduce state-independent, nonperturbativ...",Quantum Speed Limits for Leakage and Decoherence.,2015.0,Physical review letters,English
9082,9082,e93d5a8632797df8201733ff347f81e5cc3ef5a8,In this study we report on jumps in the magnet...,Interaction-induced partitioning and magnetiza...,2011.0,Physical review letters,English
9427,9427,1384974a2b790d01ffa237bd084eec0af62688f7,Quantum teleportation faces increasingly deman...,"Remote preparation of single-photon ""hybrid"" e...",2010.0,Physical review letters,English


4195 seems promising. It's quantum mechanics and according to the abstract I would put it in either a condensed matter journal or physical chemistry journal (as this kind of quantum calculation are heavily used in computational chemistry) (ie. PRL, PRE, journal of chemical physics PCCP or somthing like that)<br>
Loading the model and finding the top 500 similar abstracts:

In [86]:
from gensim.models.word2vec import Word2Vec
model= Word2Vec.load('../models/doc2vec/doc2vec_one_file_filter.model')

In [87]:
sims = model.docvecs.most_similar(df.iloc[6422].id, topn=500)
sims_id = [n for n,v in sims]

In [88]:
df_sims = df[df.id.isin(sims_id)]

In [89]:
df.iloc[6422].paperAbstract

'We introduce state-independent, nonperturbative Hamiltonian quantum speed limits for population leakage and fidelity loss, for a gapped open system interacting with a reservoir. These results hold in the presence of initial correlations between the system and the reservoir, under the sole assumption that their interaction and its commutator with the reservoir Hamiltonian are norm bounded. The reservoir need not be thermal and can be time dependent. We study the significance of energy mismatch between the system and the local degrees of freedom of the reservoir that directly interact with the system. We demonstrate that, in general, by increasing the system gap we may reduce this energy mismatch, and, consequently, drive the system and the reservoir into resonance; this can accelerate fidelity loss, irrespective of the thermal properties or state of the reservoir. This implies that quantum error suppression strategies based on increasing the gap are not uniformly beneficial. Our speed 

In [93]:
df_sims.iloc[1].paperAbstract

'Despite the considerable evidence showing that dispersal between habitat patches is often asymmetric, most of the metapopulation models assume symmetric dispersal. In this paper, we develop a Monte Carlo simulation model to quantify the effect of asymmetric dispersal on metapopulation persistence. Our results suggest that metapopulation extinctions are more likely when dispersal is asymmetric. Metapopulation viability in systems with symmetric dispersal mirrors results from a mean field approximation, where the system persists if the expected per patch colonization probability exceeds the expected per patch local extinction rate. For asymmetric cases, the mean field approximation underestimates the number of patches necessary for maintaining population persistence. If we use a model assuming symmetric dispersal when dispersal is actually asymmetric, the estimation of metapopulation persistence is wrong in more than 50% of the cases. Metapopulation viability depends on patch connectivi

In [94]:
df_sims.groupby('journalName').count().sort_values(by='id', ascending=False).head(10)

,Unnamed: 0,id,paperAbstract,title,year,lang
journalName,,,,,,
The Journal of chemical physics,26,26,26,26,26,26
"Physical review. E, Statistical, nonlinear, and soft matter physics",19,19,19,19,19,19
Physical review letters,17,17,17,17,17,17
Proceedings of the National Academy of Sciences of the United States of America,14,14,14,14,14,14
CoRR,13,13,13,13,13,13
Journal of the American Chemical Society,8,8,8,8,8,8
Journal of physics. Condensed matter : an Institute of Physics journal,7,7,7,7,7,7
Physical review. E,7,7,7,7,7,7
The journal of physical chemistry. B,7,7,7,7,7,7


PRL is there, and most of the journals are in either condensed matter or physical chemistry. There are obviosly not enough samples and using the full dataset should fix that. I also need of think of a way to normalize things by the total number of publication in a journal (i.e if there is a publication with tons of papers that are somwhat similar to our abstract it will be ranked high)

Next, lets try a random abstract from 'Human Genetics' I pulled from google:

In [95]:
abst = 'Single nucleotide polymorphisms (SNPs) constitute the bulk of human genetic variation, occurring with an average density of ∼1/1000 nucleotides of a genotype. SNPs are either neutral allelic variants or are under selection of various strengths, and the impact of SNPs on fitness remains unknown. Identification of SNPs affecting human phenotype, especially leading to risks of complex disorders, is one of the key problems of medical genetics. SNPs in protein-coding regions that cause amino acid variants (non-synonymous cSNPs) are most likely to affect phenotypes. We have developed a straightforward and reliable method based on physical and comparative considerations that estimates the impact of an amino acid replacement on the three-dimensional structure and function of the protein. We estimate that ∼20% of common human non-synonymous SNPs damage the protein. The average minor allele frequency of such SNPs in our data set was two times lower than that of benign non-synonymous SNPs. The average human genotype carries approximately 103 damaging non-synonymous SNPs that together cause a substantial reduction in fitness.'

In [96]:
abst2 = abst.lower().split(' ')

In [97]:
bb = model.infer_vector(abst2)

In [98]:
sims = model.docvecs.most_similar(positive=[model.infer_vector(abst2)], topn=5000)
sims_id = [n for n,v in sims]

In [99]:
df_sims = df[df.id.isin(sims_id)]

In [100]:
df_sims.groupby('journalName').count().sort_values(by='id', ascending=False).head(15)

,Unnamed: 0,id,paperAbstract,title,year,lang
journalName,,,,,,
Proceedings of the National Academy of Sciences of the United States of America,67,67,67,67,67,67
The Journal of biological chemistry,53,53,53,53,53,53
Genetics,29,29,29,29,29,29
Gene,27,27,27,27,27,27
Human Genetics,24,24,24,24,24,24
Biochemical and biophysical research communications,23,23,23,23,23,23
American journal of human genetics,23,23,23,23,23,23
Genome research,23,23,23,23,23,23
Nature,22,22,22,22,22,22


I have absolutly no knoladge in the field but it seems reasonable.

Lastly, I'm gonna try one of my papers:

In [101]:
abst= 'The main challenge in predicting sliding friction is related to the complexity of highly \
nonequilibrium processes, the kinetics of which are controlled by the interface temperature. \
Our experiments reveal a nonmonotonic enhancement of dry nanoscale friction at cryogenic \
temperatures for different material classes. Concerted simulations show that it emerges from \
two competing processes acting at the interface: the thermally activated formation as well as \
rupturing of an ensemble of atomic contacts. These results provide a new conceptual \
framework to describe the dynamics of dry friction'

In [102]:
abst2 = abst.lower().split(' ')
bb = model.infer_vector(abst2)
sims = model.docvecs.most_similar(positive=[model.infer_vector(abst2)], topn=5000)
sims_id = [n for n,v in sims]
df_sims = df[df.id.isin(sims_id)]
df_sims.groupby('journalName').count().sort_values(by='id', ascending=False).head(15)

,Unnamed: 0,id,paperAbstract,title,year,lang
journalName,,,,,,
The Journal of biological chemistry,115,115,115,115,115,115
Langmuir : the ACS journal of surfaces and colloids,97,97,97,97,97,97
Proceedings of the National Academy of Sciences of the United States of America,94,94,94,94,94,94
The Journal of chemical physics,92,92,92,92,92,92
Journal of the American Chemical Society,83,83,83,83,83,83
Biochemistry,81,81,81,81,81,81
Physical chemistry chemical physics : PCCP,79,79,79,79,79,79
"Physical review. E, Statistical, nonlinear, and soft matter physics",76,76,76,76,76,76
The journal of physical chemistry. B,73,73,73,73,73,73


Except from Biochemistry all of them are journals that are fit for this publication. This was actually published in PRL which is not here, but it was an exception and the novelty of the discovery is what put it there otherwise we would have send it to one of the journals above. 

So... This needs more work, but it seems like it's going in the right direction. Next I'll train a Glove model and test the mean vectors distance for similarity, there is some reaserch which suggests this can work better for short documents)